### **代码框架**

```bash
agent_ppo/
├─ agent.py
├─ algorithm/
│  └─ algorithm.py
├─ conf/
│  ├─ conf.py
│  └─ train_env_conf.toml
├─ feature/
│  ├─ feature_process/   (hero/organ 特征与归一化配置)
│  ├─ definition.py
│  └─ reward_process.py
├─ model/
│  └─ model.py
└─ workflow/
   └─ train_workflow.py

```


### **各部分作用**


- **conf/** 配置文件，配置环境情况，训练的具体情况

- **workflow/**
   - `train_workflow.py`：训练流程编排。典型过程是读取配置→`env.reset`→循环`env.step`收集样本→按`eval_interval`触发评估（评估对手可选 common_ai / selfplay / 你在平台配置的指定模型）。评估阶段平台会调用智能体接口`agent.reset / agent.exploit`对战判定能力

- **feature/**
   - `eature_process/`：把环境给的原始帧数据（`observation["frame_state"]`）加工为输入特征；官方推荐的入口形态为：`feature = hero`向量特征 + 防御塔等`organ`特征。模块内含特征提取函数与归一化（`one-hot / min-max`），并考虑镜像对称（把右上象限映射到左下），保证两边视角一致
   - `definition.py`：数据协议/枚举/字段名等定义，对应文档里的`FrameState/ActorState`字段结构。比如 `frame_state` 内会包含 `frameNo / hero_states / npc_states / frame_action`等字段
   - `reward_process.py`：回报管理。文档建议将多个子回报（密集/稀疏）加权相加，并可采用“零和风格”的增量计算（本帧与上一帧差值，双方相减），默认示例已包含 “前进 forward” 与 “tower_hp_point”。

- **model/**
   - `model.py`：PPO 的策略/价值网络定义。输出需要和层次化动作空间对齐：先决策 what（which_button），再按按钮类型决定是否输出 how（move/skill 的二维偏置）与 who（目标），并配合 Sub Action Mask / Legal Action Mask 过滤不需要或不合法的分量

- **algorithm**
   - `algorithm.py`：PPO 算法实现（剪切比目标、稳定更新）。官方文档里的监控指标包括 reward / total_loss / value_loss / policy_loss / entropy_loss，可用于观察训练是否“震荡向上”与收敛趋势。

- **agent.py**
   - 智能体对外接口的封装：在评估时平台调用`agent.reset`与`agent.exploit`完成对战；这层也会衔接特征处理、模型前向与动作后处理（掩码应用、参数拼装）。

- **环境交互要点**
   - `reset/step` 返回结构：`step` 返回 `frame_no / observation / terminated / truncated / extra_info`；`observation[agent_id]` 中含 `frame_state`（英雄/小兵/塔等集合）、`legal_action、sub_action_mask、win` 等
   - 层次化动作空间：what/how/who 三段式；动作分量有按钮、移动/技能的 X/Z 偏置（16×16）、目标（None/英雄/塔/小兵/野怪）。
   - Action Mask：
      - Sub Action Mask：依据按钮类型决定是否需要方向/目标输出；
      - Legal Action Mask：屏蔽冷却中或规则不允许的动作，减少无效探索
   - Step 与帧的映射：1 step = 6 frames；每一步执行相同动作跨 6 帧，利于稳定观测与反馈。
   - 监控指标：对局结束时记录 `win_rate / tower_hp / frame / money_per_frame / K/D / hurt_per_frame`，训练监控面板的 `env` 模块会展示。

### **特征处理**

特征处理部分全部放在了`feature/`文件夹的`feature/feature_process`内部，具体文件如下：

> agent_ppo/feature/feature_process/feature_normalizer.py
> agent_ppo/feature/feature_process/hero_process.py
> agent_ppo/feature/feature_process/organ_process.py
> agent_ppo/feature/feature_process/hero_feature_config.ini
> agent_ppo/feature/feature_process/organ_feature_config.ini

其中`feature_normalizer.py`用于进行归一化处理，方便传入网络进行计算，`hero_feature_config.ini`和`organ_feature_config.ini`用于存储英雄/防御塔的相应特征信息，`hero_process.py`和`organ_process.py`分别用于处理英雄/防御塔的特征信息

#### **归一化文件**

归一化文件为`feature_normalizer.py`文件，提供`one_hot`编码和`min_max`两种归一化方式

1. `one_hot`归一化方式
```python
def one_hot(self, value, value_list, compare_method):
    # one hot encoding
    if compare_method != "eq":
        raise ValueError("Unsupported compare method: " + compare_method)
    return [1 if value == v else 0 for v in value_list]
```

传入参数：`value`原始取值，`value_list`参与`one_hot`的备选值序列，`compare_method`比较方式(当前实现函数中只支持eq)

返回内容：若`compare_method!=eq`直接抛出错误，其他返回一个长度为`value_list`的0/1列表，当且仅当`valuse_list`中的值等于`value`时取1，其余取0

示例：`one_hot(2, [0,1,2], 'eq') -> [0,0,1]`

2. `min_max`归一化方式
```python
def min_max(self, value, min_value, max_value):
    # Normalize the value based on min and max values
    if value <= min_value:
        return 0
    elif value >= max_value:
        return 1
    else:
        return (value - min_value) / (max_value - min_value)
```

输入参数：数值`value`，下界/上界`min_value,max_value`

返回内容：如果`value <= min_value`则返回0，如果`value >= max_value`则返回1，其余返回`(value - min_value) / (max_value - min_value)`


文档中配有相应的归一化方式的判断使用函数`parse_config(self, config_list)`传入一个配置列表。随后进行解析分割，`parts[0]`作为特证名`feature_name`，`parts[1]`作为方法`method`，根据不同的方法，分割具体数据调用相应的归一化函数。其中传入的`config_list`可以参考两个.ini文件



#### **初始化文件**

定义了类`FeatureProcess`，用于进行综合调用`hero`和`organ`的特征处理，初始化函数有两个：

1. `__init__(self, camp)`

按照当前我方阵营`camp`构造一套可用的特征处理(构造hero特征处理器和organ特征处理器)，注意后序进行镜像变换

2. `reset(self, camp)`

同样的类似`__init__`，当阵营发生变化时进行重构

完成初始化/重构后，使用`process_organ_feature(self, frame_state)`和`process_hero_feature(self, frame_state)`产生二者的拼接向量，最后调用`process_feature(self, observation)`将二者的向量按顺序拼接输出模型


### **英雄特征处理**

